In [1]:
import jupyros as jr
import rospy
import tf
import numpy as np
import ipywidgets as widgets
import math

from geometry_msgs.msg import PoseStamped
from tf.transformations import euler_from_quaternion
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

from robotcontroller import RobotController

rospy.init_node('jupyter_node')

In [2]:
jr.publish('/cmd_vel', Twist)

In [3]:
mapframe = 'odom'
transformlistener = tf.TransformListener()
cmdvelpub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

robotcontroller = RobotController(transformlistener, mapframe, cmdvelpub)
jr.subscribe('/odom', Odometry, robotcontroller.newodometry)

In [4]:
output = widgets.Output()

def slider_change(change):
    with output:
        newvalue = change['new']
        print(newvalue)    
        robotcontroller.appendbehavior(RotateToState(newvalue))
    return

slider = widgets.FloatSlider(
    value=1,
    min=-math.pi,
    max=math.pi,
    step=0.1,
    description='Target theta:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)
slider.observe(slider_change, names='value')

display(slider, output)

FloatSlider(value=1.0, continuous_update=False, description='Target theta:', max=3.141592653589793, min=-3.141…

Output()

In [5]:
from rotatetostate import RotateToState
from drivetoposition import DriveToPosition

def navgoal(message):

    if robotcontroller.latestodominmapframe:
        
        navgoalinframe = transformlistener.transformPose(mapframe, message)
        
        dx = navgoalinframe.pose.position.x - robotcontroller.latestodominmapframe.pose.position.x
        dy = navgoalinframe.pose.position.y - robotcontroller.latestodominmapframe.pose.position.y
        
        yaw = math.atan2(dy, dx)
        robotcontroller.appendbehavior(DriveToPosition(navgoalinframe))
        robotcontroller.appendbehavior(RotateToState(yaw))

    return

jr.subscribe('/move_base_simple/goal', PoseStamped, navgoal)

         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-tf2/src/work/src/buffer_core.cpp
         at line 278 in /opt/conda/build_artifacts/ros-noetic-tf2_1615118253106/work/ros-noetic-t